In [172]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import requests
from bs4 import BeautifulSoup

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from copy import deepcopy

stop_words = stopwords.words('english')
recquired_length = 0.33333
total_length = 0

In [173]:
word_embeddings = {}
def load_word_embeddings():
    f = open("/media/tatan/A0A04E3AA04E16E6/word_vectors/glove.6B.100d.txt",encoding = 'utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

In [174]:
load_word_embeddings()

In [210]:
html = requests.get("https://medium.com/sciforce/towards-automatic-text-summarization-extractive-methods-e8439cd54715").text
soup = BeautifulSoup(html,'html.parser')

In [211]:
title = soup.find('title').text
paras = soup.findAll('p')
text = [i.text for i in paras if len(i.text)>70]
text = "".join(text)

In [214]:
sentence_tokens = sent_tokenize(text)
total_length = len(sentence_tokens)

In [215]:
def sentencevector_avg(word_vectors):
    avg_vector = []
    if len(word_vectors)!=0:
        avg_vector = np.array([sum([vector[axis] for vector in word_vectors])/len(word_vectors) for axis in range(len(word_vectors[0]))])
    else:
        avg_vector = np.zeros((100,))
    return avg_vector

In [216]:
def sentencevector_avg_weighted(word_vectors,tmp_vectors,weight):
    for i in range(len(tmp_vectors)):
        for j in range(len(tmp_vectors[0])):
            tmp_vectors[i][j] = weight*tmp_vectors[i][j]
    word_vectors+=tmp_vectors
    avg_vector = []
    if len(word_vectors)!=0:
        avg_vector = np.array([sum([vector[axis] for vector in word_vectors])/len(word_vectors) for axis in range(len(word_vectors[0]))])
    else:
        avg_vector = np.zeros((100,))
    return avg_vector

In [217]:
def remove_stopwords_and_punctuations(sen):
    tokenizer = RegexpTokenizer(r'\w+')
    sen = " ".join(tokenizer.tokenize(sen))
    new_vec = " ".join([word for word in sen.split() if word not in stop_words])
    return new_vec

In [218]:
def get_word_vectors(sentence):
    sentence = remove_stopwords_and_punctuations(sentence)
    word_vectors = []
    for word in sentence.split():
        try:
            word_vectors.append(deepcopy(word_embeddings[word.lower()]))
        except KeyError as e:
            print(e)
    return word_vectors

In [219]:
def getSummary(title,sentence_tokens):
    summary = []
    title_vector = sentencevector_avg(get_word_vectors(title))
    for sentence_token in sentence_tokens:
        sentence_vector = []

        sentence_vector = sentencevector_avg(get_word_vectors(sentence_token))
        
        tmp = pearsonr(sentence_vector,title_vector)[0]
        summary.append((tmp,sentence_token))
        title_vector = sentencevector_avg_weighted(get_word_vectors(title),get_word_vectors(sentence_token),0.08)
            
    return summary

In [ ]:
summary = [(i,s) for i,s in enumerate(getSummary(title,sentence_tokens))]
summary.sort(key = lambda x: x[1][0], reverse = True)

final_summary = sorted(summary[:int(recquired_length*total_length)],key = lambda x: x[0])

summary = ""
for i in final_summary:
    summary+=i[1][1]+"\n"
print(summary,len(summary))